In [245]:
import datetime
import numpy as np
import datetime
import pandas as pd
from datetime import timedelta
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

epoch = datetime.datetime.utcfromtimestamp(0)
def unix_time_millis(dt):
    return int(round((dt - epoch).total_seconds() * 1000.0, 0))


In [246]:
prices = pd.read_csv ('latest_prices.csv')
max_price_date = prices.Date.max()

In [247]:
prices[(prices['Stock Name'].str.startswith('Berger')) ]

,UnixDate,Price,Date,1d_change,5d_change,10d_change,20d_change,30d_change,50d_change,100d_change,150d_change,200d_change,250d_change,Stock Name,ticker_id
60,1586447940000,474.3,2020-04-09,0.023301,-0.047495,0.116263,-0.100256,-0.165406,-0.154018,-0.026678,0.281372,0.483578,0.429045,Berger Paints India,488.0


In [248]:
file_name = 'vro_revenues.csv'
all_data = pd.read_csv(file_name)
all_data = all_data[~all_data['Stock Name'].str.startswith('S&P')]

In [249]:
all_data['1y_change'] = all_data.sort_values('Date').groupby(['Stock Name', 'name']).value.pct_change(1)
all_data['2y_change'] = all_data.sort_values('Date').groupby(['Stock Name', 'name']).value.pct_change(2)
all_data['3y_change'] = all_data.sort_values('Date').groupby(['Stock Name', 'name']).value.pct_change(3)
all_data = all_data[all_data['1y_change']>0]
all_data = all_data[all_data['2y_change']>0]
all_data = all_data[all_data.name != 'RoE']
stocks_with_positive_cash_flow = all_data[(all_data.name=='Cash From Operations') & (all_data.value > 0)]['Stock Name'].unique().tolist()
all_data = all_data[all_data['Stock Name'].isin(stocks_with_positive_cash_flow)]
all_data.dropna(inplace=True)
all_data.shape

(2994, 9)

In [250]:
all_data[all_data['Stock Name'].str.startswith('Berg') ]

,Date,Stock Name,name,ticker_id,value,yearly_change,1y_change,2y_change,3y_change
18384,2019-03-01,Berger Paints India,Revenue,488.0,6061.86,0.173476,0.173476,0.331618,0.435405
18385,2019-12-01,Berger Paints India,Revenue,488.0,6483.07,0.069485,0.069485,0.255015,0.424146
18389,2019-03-01,Berger Paints India,Earnings,488.0,5.13,0.080000,0.080000,0.051230,0.342932
18390,2019-12-01,Berger Paints India,Earnings,488.0,6.86,0.337232,0.337232,0.444211,0.405738
63079,2019-03-01,Berger Paints India,Cash From Operations,488.0,561.67,0.331287,0.331287,0.423715,-0.055334


In [251]:
max_revenue = all_data[all_data.name=='Revenue'].value.max()
max_cash = all_data[all_data.name=='Cash From Operations'].value.max()
max_vro_cash_flow_date = all_data[all_data.name == 'Cash From Operations']['Date'].max()
max_vro_revenue_date = all_data[all_data.name == 'Revenue']['Date'].max()
max_cash, max_revenue, max_vro_cash_flow_date

(71459.0, 611934.0, '2019-12-01')

In [252]:
@interact
def filter_cash_flow(cash_flow_change = (2,10), revenue_1y_change = (2,10), min_revenue = (4000, 15000, 500), discount=(-20, 20, 2)):
    stocks_by_cash_change = all_data.loc[(all_data.name == 'Cash From Operations') & (all_data['1y_change'] > cash_flow_change/100) & (all_data.Date >= '2019-03-01') ]['Stock Name'].unique().tolist()
    stocks_by_rev_change = all_data.loc[(all_data.name == 'Revenue') & (all_data['1y_change'] > revenue_1y_change/100) & (all_data.Date >= '2019-12-01')]['Stock Name'].unique().tolist()
    stock_with_min_revenue = all_data.loc[(all_data.name == 'Revenue') & (all_data.value >= min_revenue)]['Stock Name'].unique().tolist()
    stocks = set(stocks_by_cash_change).intersection(set(stocks_by_rev_change)).intersection(set(stock_with_min_revenue))
    df= all_data.loc[(all_data['Stock Name'].isin(stocks)) & (all_data.Date == max_vro_revenue_date) ]
    df = df.loc[(df.name == 'Revenue')]
    df = df.merge(prices, on ='Stock Name', how='left')
    df = df.drop(['100d_change', '200d_change', '150d_change', 'yearly_change', 'value', '3y_change', 'Date_x', 'ticker_id_x', 'ticker_id_y', 'UnixDate'], axis=1)
#     Show stocks fallen down by 
    df = df[df['30d_change']<discount/100].drop_duplicates()
    return df.sort_values(['30d_change'])


interactive(children=(IntSlider(value=6, description='cash_flow_change', max=10, min=2), IntSlider(value=6, de…

In [243]:
print("------- END ---------")

------- END ---------
